# Pruebas de Ruido con S.T. sintéticas

Los decibeles expresan una relación de poder, no una cantidad. Dicen cuántas veces más (dB positivo) o menos (dB negativo) pero no cuánto en términos absolutos. Los decibeles son logarítmicos, no lineales. Por ejemplo, 20 dB no es el doble de la relación de potencia de 10 dB.

La ecuación para decibelios es:

$A = 10 * log10 (P2 / P1) (dB)$

donde P1 es la potencia que se mide y P1 es la referencia con la que se compara P2.
Para convertir de la medida de decibelios a la relación de potencia:

$P2 / P1 = 10^{(A / 10)}$

In [1]:
import os, sys
sys.path.insert(0, '/home/victort/Documentos/SEMESTRE 17-18 MAESTRIA/TESIS/Deteccion Outliers')
import outliersDetection as od
import requests, pandas as pd, numpy as np
import time, json
import bokeh
from scipy import signal, fft
from statsmodels.tsa.stattools import adfuller, acf, pacf
import statsmodels.graphics as smg
%matplotlib inline
from bokeh.plotting import gridplot, figure, output_file, output_notebook, show
from datetime import datetime
from __future__ import print_function
from PyAstronomy import pyasl
from pylab import *
# Biblioteca para calcular los exponentes de Lyapunov
import nolds 
import datetime

None


## Valores faltantes

In [2]:
def valoresFaltantes(data):
    tam = len(data)
    num_nans = round(round(data.isnull().sum())/float(tam)*100) 
#     print("Porcentaje de valores faltantes",num_nans,"tamano de la serie de tiempo",tam)
    return num_nans

## Outliers

In [3]:
def calcularOutliers(nombre,original=0,faltantes=0,outliers=0,SNR_dB=0):
    ts = pd.DataFrame()
    tam = 0
    if original == 1:
        filename = "st originales/"+nombre+".dat"
        ts_aux = pd.read_csv(filename)
        ts_aux = ts_aux.dropna()
        tam = len(ts_aux)
        if tam > 20000:
            ts_aux = ts_aux[0:20000]
        ts = pd.DataFrame({'col0': np.arange(len(ts_aux)),'col1': ts_aux['col1']})
    else:
        filename = "st perturbadas/"+nombre+"/"+nombre+"_"+str(faltantes)+"_"+str(outliers)+"_"+str(SNR_dB)+".csv"
        ts = pd.read_csv(filename)
        ts = ts.dropna()
        tam = len(ts)
        if tam > 20000:
            ts = ts[0:20000]
#         print("TSSSS\n",ts)
#     print(ts)
    Inliers,Outliers,Pos = od.obtenerOutliers(ts,k=5,showPlot=1)
#     print(len(Outliers[1]))
    tam = len(ts)
    totalOutliers = round(len(Outliers)/float(tam)*100)
#     print("totalOutliers",totalOutliers)
    return totalOutliers
# calcularOutliers("duffing_oscillator",original=1)

## SNR

In [4]:
def obtener_SNR(xn,x=[],t=[],filename='SNR.csv',showPlot=0):
    """
    Parameters:
           xn.- Signal + noise
           x.- Original signal
           t.- time
           filename.- donde guardar los SNR obtenidos
           showPlot.- muestra las graficas con 1 y con 0 no
    Return:
           SNR.- Signal to noise ratio 
    """
    # Estimate noise using robust estimate
    beq = pyasl.BSEqSamp()
    # Define order of approximation (use larger values such as 2,3, or 4 for
    # faster varying or less well sampled data sets; also 0 is a valid order)
    N = 2
    # Define 'jump parameter' (use larger values such as 2,3, or 4 if correlation
    # between adjacent data point is suspected)
    j = 1
    nstd, nstdstd = beq.betaSigma(xn['col1'], N, j, returnMAD=True)
    # Obteniendo la varianza apartir de la desviacion estandar
    var_noise = np.power(nstd,2)
    # Ma va a contener la senal
    Ma = []
    # Res contiene los residuos los cuales se consideran como el ruido
    Res = []
    SNRcalculado = 0
    for window in range(2,20):
#         print("Ancho de ventana = ",window)
        # Moving Average Medias Moviles
        Ma = xn['col1'].rolling(window=window,center=True).mean()
        # Residuos de la Media movil
        Res = xn['col1'] - Ma
        # Quitando los 
        Res = Res.dropna()
        # Obteniendo la varianza del ruido
        varianza_Noise_sin = np.var(Res)      
#         restd = np.std(Ma)
        # Obteniendo las frecuencias y el espectro de potencia o densidad espectral de potencia
        fsenal, Pxx_den_senal = signal.periodogram(Ma[~np.isnan(Ma)])
        fnoise, Pxx_den_noise = signal.periodogram(Res[~np.isnan(Res)])
        # Quitando nans de la densidad espectral de potencia tanto para la senal como para el ruido
        Pxx_den_senal = Pxx_den_senal[~np.isnan(Pxx_den_senal)]
        Pxx_den_noise = Pxx_den_noise[~np.isnan(Pxx_den_noise)]

        #Si la diferencia entre la varianza de pyastronomy y la varianza del ruido extraido es menor a 1e-3
        #Podemos decir que se ha encontrado el SNR o que no contiene SNR
        if np.abs(var_noise - varianza_Noise_sin) < 1e-3:
            # Si la ventana es de 2 quiere decir que la serie no contiene ruido
            if window == 2:
                print("No contiene ruido")
                filename.write(",,No hay ruido\n")
                SNRcalculado = "No hay ruido"
                break
            # Esta x se utilizo para las pruebas
            if len(x) > 0:
                SNRcalculado = 10*np.log10(np.mean(Pxx_den_senal)/np.mean(Pxx_den_noise))
                print("\nSe redujo el ruido con una ventana de",window)
                print("SNR detectado",SNRcalculado)
#                 print("SNR mediaS/desvN",1/(np.mean(Ma)/np.std(Res)))
                filename.write(str(SNRcalculado)+'\n')
                if showPlot:
                    od.grafica2D(t,Ma,t,x,
                             title1='Ruido reducido',title2='Original')
                    od.grafica2D(t,Ma,t,xn['col1'],
                             title1='Ruido reducido',title2='Ruido')
            else:
                # Se calcula el SNR = Psignal/Pnoise donde P es la potencia media https://en.wikipedia.org/wiki/Signal-to-noise_ratio
                SNRcalculado = 10*np.log10(np.mean(Pxx_den_senal)/np.mean(Pxx_den_noise))
                print("\nSe redujo el ruido con una ventana de",window)
                print("SNR detectado",SNRcalculado)
                filename.write(str(SNRcalculado)+'\n')
#                 print("SNR mediaS/desvN",1/(np.mean(Ma)/np.std(Res)))
                if showPlot:
                    od.grafica2D(t,Ma,t,xn['col1'],
                             title1='Noise reduced',title2='Noise signal')
            break
        if window == 19:
            SNRcalculado = 10*np.log10(np.mean(Pxx_den_senal)/np.mean(Pxx_den_noise))
            filename.write(str(SNRcalculado)+',,,Max\n')
    return SNRcalculado


#Funcion para graficar el periodograma
# import matplotlib.pyplot as plt
def plot_periodogram(df_x,title,legend,showPlot=0):
    #Obtenemos la frecuencia y densidad espectral de potencia o espectro de potencia.
    f, Pxx_den = signal.periodogram(df_x['col1'].T)
    Pxx_den = np.ravel(Pxx_den)
    Pxx_den[0] = np.nan
    #print(Pxx_den[0:10])
    if showPlot:
        p = bokeh.plotting.figure(title=title, y_axis_type="log",
               background_fill_color="#fafafa")
        p.line(x=f, y=np.ravel(Pxx_den), legend=legend, line_color="blue")
        bokeh.plotting.show(p)
    return f, Pxx_den

def variacion(Pxx_den):
    # Esta funcion recibe el espectro de potencia
    # El indice es el del elemento mas grande
    indice = np.argmax(Pxx_den[~np.isnan(Pxx_den)])
    # Si el indice se encuentra delante del 10% de la posicion quiere decir que no se puede calcular el SNR
    if indice > len(Pxx_den)*0.1:
        return False
    else:
        return True

def plotseries(tipo,path="",ts=[]):
    # Funcion para plotear las series
    if tipo == 0:
        od.grafica2D(np.arange(len(ts)),ts,title1='TS')
    else:
        ts = pd.read_csv(path)
#         ts = ts.dropna()
        od.grafica2D(np.arange(len(ts)),ts['col1'],title1='TS')

In [6]:
def SNRTsPruebas(nombre,prueba=0,SNR_dB=-1,showPlot=0):
    path = "st con ruido/"+nombre+str(SNR_dB)
    numeroDeSeries = 1
    if prueba:
        numeroDeSeries = 10
    SNRObtenido = 0
    if SNR_dB == -1:
        print("entra",os.path.exists)
        if not os.path.exists:
            print("entra")
            os.mkdir(path)
        numeroDeSeries = 1
    filename = path+"/snr "+nombre+str(SNR_dB)+".csv"
    csv_file = open(filename, "w")  
    columnTitleRow = "SNR detectado, SNR no detectado, No contiene SNR, Prueba con un SNR = "+str(SNR_dB)+'\n'
    csv_file.write(columnTitleRow)
    tiempoGlobal = time.time()
    for i in range(0,numeroDeSeries):
        tiempoInicioTarea = time.time()
        print("i",i)
        if SNR_dB != -1:
            ts = pd.read_csv("st con ruido/"+nombre+str(SNR_dB)+"/ts "+str(i)+".csv")
        else:
            #Read the file 
            ts = pd.read_csv("st originales/"+nombre+".dat")
            ts = ts[0:20000]
            ts = ts.dropna()
#         od.grafica2D(np.arange(len(ts)),ts['col1'],title1=nombre+" SNR "+str(SNR_dB))    
        f_ts, Pxx_den_ts = plot_periodogram(ts,nombre,nombre,showPlot=showPlot)
        varia = variacion(Pxx_den_ts)
        if varia:
            #Obtener el SNR
            SNRObtenido = obtener_SNR(ts,t=np.arange(len(ts)),filename=csv_file,showPlot=showPlot)
        else:
            print("Varia rapidamente")
            SNRObtenido = "No se puede calcular SNR"
            csv_file.write(",No se puede calcular SNR,\n")
        tiempoFinalTarea = time.time()
        print("Tiempo total ",tiempoFinalTarea-tiempoGlobal,"Tiempo tarea ",tiempoFinalTarea-tiempoInicioTarea)
    columnTitleRow = "=PROMEDIO(A2:A11)\n"
    csv_file.write(columnTitleRow)
    csv_file.close()
    return SNRObtenido

In [7]:
def SNRTs(nombre,faltantes=0,outliers=0,SNR_dB=0,showPlot=0):
    tiempoGlobal = time.time()
    path = "st perturbadas/"+nombre+"/"+nombre+"_"+str(faltantes)+"_"+str(outliers)+"_"+str(SNR_dB)+".csv"
    SNRObtenido = 0
    ts = pd.read_csv(path)
    if len(ts) > 20000:
        ts = ts[0:20000]
    ts = ts.dropna()
    
    filename = "snr/"+nombre+str(SNR_dB)+".csv"
    csv_file = open(filename, "w")  
    columnTitleRow = "SNR detectado, SNR no detectado, No contiene SNR, Prueba con un SNR = "+str(SNR_dB)+'\n'
    csv_file.write(columnTitleRow)
    
#     ts = ts.dropna()
#     od.grafica2D(np.arange(len(ts)),ts['col1'],title1=nombre+" SNR "+str(SNR_dB))    
    f_ts, Pxx_den_ts = plot_periodogram(ts,nombre,nombre,showPlot=showPlot)
    varia = variacion(Pxx_den_ts)
    print("varia",varia)
    if varia:
        #Obtener el SNR
        SNRObtenido = obtener_SNR(ts,t=np.arange(len(ts)),filename=csv_file,showPlot=showPlot)
    else:
#         print("Varia rapidamente")
        SNRObtenido = "No se puede calcular SNR"
    csv_file.close()
    tiempoFinalTarea = time.time()
    print("Tiempo total SNRTs",tiempoFinalTarea-tiempoGlobal)
    return SNRObtenido

In [17]:
# print(calcularOutliers("lorenz",original=0,faltantes=0,outliers=5,SNR_dB=0))
# plotseries(1,path="st perturbadas/halvorsen_attractor/halvorsen_attractor_10_25_0.csv")
windAristeoM10m = pd.read_csv('st perturbadas/seno/seno_0_5_0.csv')

od.obtenerOutliers(windAristeoM10m,k=5,showPlot=1)
# plotseries(1,path="st originales/henon.dat")
# print(SNRTsPruebas("henon",prueba=0,SNR_dB=-1,showPlot=1))
# SNRTs("henon",0,0,0,showPlot=1)

Loading BokehJS ...

Datos analizados: 200 
Inlliers: 195 
Outliers 5
Media 1.002672570083664 
Dst 0.021310874318254396 
Dif 0.16387222642580523 
Donde estan [Series([], Name: col0, dtype: int64), Series([], Name: col1, dtype: float64)] 
n_neighbors_ 5 
Indice LOF [1.1665448  1.09458799 1.0945498  0.99563283 0.95423976 0.98757327
 1.00007447 1.00007623 1.00007679 1.00007613 1.00007428 1.00007125
 1.0000671  1.00006188 1.00005568 1.00004861 1.00004076 1.00003226
 1.00002326 1.00001388 1.00000429 0.99999647 0.99999241 0.99999402
 0.99999202 0.99998831 0.99999202 0.99999402 0.99999241 0.99999647
 1.00000429 1.00001388 1.00002326 1.00003226 1.00004076 1.00004861
 1.00005568 1.00006188 1.0000671  1.00007125 1.00007428 1.00007613
 1.00007679 1.00007623 1.00007447 1.00007154 1.00006748 1.0000642
 1.0000636  1.00005648 1.00003614 1.00003919 1.00003768 0.9987216
 0.99735627 0.99725692 1.00282809 1.00146992 1.00142927 1.00279282
 1.00415517 0.99730809 0.9930832  0.98877086 0.98863354 1.01170192
 1.00589517 1.0058224

(     col0      col1
 0       0  0.000000
 1       1  0.062791
 2       2  0.125333
 3       3  0.187381
 4       4  0.248690
 5       5  0.309017
 6       6  0.368125
 7       7  0.425779
 8       8  0.481754
 9       9  0.535827
 10     10  0.587785
 11     11  0.637424
 12     12  0.684547
 13     13  0.728969
 14     14  0.770513
 15     15  0.809017
 16     16  0.844328
 17     17  0.876307
 18     18  0.904827
 19     19  0.929776
 20     20  0.951057
 21     21  0.968583
 22     22  0.982287
 23     23  0.992115
 24     24  0.998027
 25     25  1.000000
 26     26  0.998027
 27     27  0.992115
 28     28  0.982287
 29     29  0.968583
 ..    ...       ...
 170   170 -0.951057
 171   171 -0.968583
 172   172 -0.982287
 173   173 -0.992115
 174   174 -0.998027
 175   175 -1.000000
 176   176 -0.998027
 177   177 -0.992115
 178   178 -0.982287
 179   179 -0.968583
 180   180 -0.951057
 181   181 -0.929776
 182   182 -0.904827
 183   183 -0.876307
 184   184 -0.844328
 185   185 -0

## Caos

In [ ]:
def calcularCaosPrueba(nombre,SNR_dB=-1):
    # Funcion para calcular el Caos usando la biblioteca nolds y el algoritmo de Rosenstein
#     tiempoInicial = time.time()
    filename = ""
    # Revisando si la st está contaminada con ruido o es la original
    if SNR_dB != -1:
        filename = "st con ruido/"+nombre+str(SNR_dB)+"/ts 0"+".csv"
    else:
        filename = "st originales/"+nombre+".dat"
    # Se carga la ts
    ts = pd.read_csv(filename)
    tamano = len(ts)
    # Se limita a que a lo maximo el tamano de la ts sea 20000 
    if tamano > 20000:
        tamano = 20000
    ts = ts[0:tamano]
#     print("TS\n\n",ts['col1'].shape)
    # Se calcula el exponente
    expo = nolds.lyap_r(np.ravel(ts['col1']))
    #expo = max(nolds.lyap_e(ts['col1']))
#     print("Tiempo total ",time.time()-tiempoInicial)
    return expo

def calcularCaos(nombre,faltantes=0,outliers=0,SNR_dB=0,showPlot=0):
    tiempoGlobal = time.time()
    path = "st perturbadas/"+nombre+"/"+nombre+"_"+str(faltantes)+"_"+str(outliers)+"_"+str(SNR_dB)+".csv"
    ts = pd.read_csv(path)
    if len(ts) > 20000:
        ts = ts[0:20000]
    ts = ts.dropna()
    expo = nolds.lyap_r(np.ravel(ts['col1']))
    #expo = max(nolds.lyap_e(ts['col1']))
#     print("Tiempo total ",time.time()-tiempoInicial)
    return expo

## Obtener caracteristicas

In [ ]:
def obtenerCaracteristicas(nombreTs,showPlot=0):
    niveles = [0,5,10]
    tam = 0
    faltantes = 0
    outliers = 0
    snr = 0
    caos = 0
    
    tiempoInicial = time.time()
    #####
    
    filename = "caracteristicas/"+nombreTs+str(datetime.datetime.now())+".csv"
    csv_file = open(filename, "w")
    columnTitleRow = "Tamano ts,Va_Fa,Out,SNR_In,Valores Faltantes,Outliers,SNR,Caos\n"
    csv_file.write(columnTitleRow)    
    for i in niveles:
        for j in niveles:
            for k in niveles:
                print(i,j,k)
                # filename = "st con ruido/seno5/ts 0.csv"
                filename = "st perturbadas/"+nombreTs+"/"+nombreTs+"_"+str(i)+"_"+str(j)+"_"+str(k)+".csv"
                ts_aux = pd.read_csv(filename)
                ts_aux = ts_aux['col1']
                tam = len(ts_aux)
                if tam > 20000:
                    tam = 20000
                    ts_aux = ts_aux[0:20000]
                # Obtener los valores faltantes
                #if i != 0:
                faltantes = valoresFaltantes(ts_aux)
                # Obtener los outliers
                #if j != 0:
                outliers = calcularOutliers(nombreTs,original=0,faltantes=i,outliers=j,SNR_dB=k)
                #if k != 0:                    
                snr = SNRTs(nombreTs,i,j,k,showPlot=0)
                caos = calcularCaos(nombreTs,i,j,k,showPlot=0)
                csv_file.write(str(tam)+","+str(i)+","+str(j)+","+str(k)+","+str(faltantes)+","+str(outliers)+","+str(snr)+","+str(caos)+'\n')
                # Ruta donde se va a guardar la ts
    csv_file.close()

    print("Tiempo total ",time.time()-tiempoInicial)

In [ ]:
SC = ["seno","henon","lorenz","rossler","chen_system","duffing_oscillator","halvorsen_attractor",
      "rucklidge_attractor","shawn_van_der_pol_oscillator","simplest_cubic_flow","simplest_linear_flow"]
obtenerCaracteristicas(SC[1],showPlot=0)

### Caos

In [ ]:
#ACT_attractor,"chen_system","duffing_oscillator","halvorsen_attractor","rucklidge_attractor","shawn_van_der_pol_oscillator","simplest_cubic_flow","simplest_linear_flow"
SC = ["seno","henon","lorenz","rossler","ACT_attractor","chen_system","duffing_oscillator","halvorsen_attractor","rucklidge_attractor","shawn_van_der_pol_oscillator","simplest_cubic_flow","simplest_linear_flow"]
# SC = ["henon"]
SNR_pasos = np.arange(4,5)
print(SNR_pasos)
for i in SC:
#     filename = "caos/"+i+" caos"+str(tamano)+".csv"
#     csv_file = open(filename, "w")
#     columnTitleRow = "Caos"+str(tamano)+"\n"
#     csv_file.write(columnTitleRow)
    caos = 0
    filename = "caos/"+i+" caos.csv"
    csv_file = open(filename, "w")
    for j in SNR_pasos:
        if j == 4:
            caos = calcularCaos(i)
            csv_file.write("0,"+str(caos)+'\n')
        else:
            caos = calcularCaos(i,j)
            csv_file.write(str(j)+","+str(caos)+'\n')
#         print(i,j,caos)
    csv_file.close()

### Periodograma

Cualquier serie de tiempo puede expresarse como una combinación de ondas de coseno (o seno) con diferentes períodos (cuánto tiempo lleva completar un ciclo completo) y amplitudes (valor máximo / mínimo durante el ciclo). Este hecho se puede utilizar para examinar el comportamiento periódico (cíclico) en una serie de tiempo.

Un periodograma se utiliza para identificar los períodos dominantes (o frecuencias) de una serie de tiempo. Esta puede ser una herramienta útil para identificar el comportamiento cíclico dominante en una serie, especialmente cuando los ciclos no están relacionados con la estacionalidad mensual o trimestral más común.


En el área de series de tiempo llamada análisis espectral, vemos una serie de tiempo como una suma de ondas de coseno con amplitudes y frecuencias variables. Uno de los objetivos de un análisis es identificar las frecuencias (o períodos) importantes en la serie observada. Una herramienta de partida para hacer esto es el periodograma. El periodograma grafica una medida de la importancia relativa de los posibles valores de frecuencia que podrían explicar el patrón de oscilación de los datos observados.

Supongamos que hemos observado datos en n puntos temporales distintos y, por conveniencia, suponemos que n es par. Nuestro objetivo es identificar frecuencias importantes en los datos. Para continuar con la investigación, consideramos el conjunto de posibles frecuencias $ω_{j} = j/n$ para $j=1, 2, ..., n/2$. Estas se llaman las frecuencias armónicas.

Representaremos las series de tiempo como

$x_{t} = \sum^{n/2}_{j=1}[β_{1} (j_{n}) cos(2πω_{j}t) + β_{2}(j_{n}) sen (2πω_{j}t)]$

Esta es una suma de las funciones de seno y coseno en las frecuencias armónicas. La forma de la ecuación proviene de la identidad dada anteriormente.

Piense en los parámetros $β_{1}(j/n)$ y $β_{2}(j/n)$ como parámetros de regresión. Luego hay un total de n parámetros porque permitimos que j se mueva de $1$ a $n/2$. Esto significa que tenemos n puntos de datos y n parámetros, por lo que el ajuste de este modelo de regresión será exacto.

El primer paso en la creación del periodograma es la estimación de los parámetros $β_{1}(j/n)$ y $β_{2}(j/n)$. Realmente no es necesario llevar a cabo esta regresión para estimar los parámetros $β_{1}(j/n)$ y $β_{2}(j/n)$. En su lugar, se utiliza la Transformada Rápida de Fourier (FFT).

Una vez estimados los parámetros, definimos


https://newonlinecourses.science.psu.edu/stat510/node/71/

In [31]:
from psr import psr
from psr import methods
import pandas as pd

In [25]:
def crearbd(nombreTs):
    niveles = [0,5,10,15,20,25]
    filename = "parametrosbd/"+nombreTs+str(datetime.datetime.now())+".csv"
    csv_file = open(filename, "w")
    columnTitleRow = "Tamano ts,Va_Fa,Out,SNR_In,m,tau,tiempo,\n"
    csv_file.write(columnTitleRow)    
    csv_file.close()    
    tiempoGlobal = time.time()
    for i in niveles:
        for j in niveles:
            for k in niveles:
                tiempoPorSerie = time.time()
                with open(filename, 'a') as csv_file:
                    writer = csv.writer(csv_file)
    #                 nameTs = "simplest_linear_flow"
                    filename = "st perturbadas/"+nameTs+"/"+nameTs+"_"+str(i)+"_"+str(j)+"_"+str(k)+".csv"
                    ts = pd.read_csv(filename)
                    ts = ts['col1']
                    dm = methods.DeterministicMethod(ts)
                    m, tau = dm.deterministic_method(2, 7)
                    p = psr.PSR(ts = np.array(ts), m= m, tau=tau)
                    db = p.get_database(path="st perturbadas/"+nameTs+"/"+nameTs+"_"+str(i)+"_"+str(j)+"_"+str(k)+'_bd.csv')
                    row = [len(ts),i,j,k,m,tau]
                    writer.writerow(row)
#                     csv_file.write(str(tam)+","+str(i)+","+str(j)+","+str(k)+","+str(faltantes)+","+str(outliers)+","+str(snr)+","+str(caos)+'\n')
                    # Ruta donde se va a guardar la ts
                csv_file.close()
                print("Tiempo por serie",time.time()-tiempoPorSerie)
    print("termino tiempo total",time.time()-tiempoGlobal)
                    # chen_system                  m = 6, tau=1, 38.175167083740234
                    # duffing_oscillator           m = 6, tau=1, 29.691254377365112s
                    # halvorsen_attractor          m = 6, tau=1, 31.72951078414917
                    # henon                        m = 6, tau=1, 30.108326196670532
                    # lorenz                       m = 6, tau=17, 37.301207065582275
                    # rossler                      m = 6, tau=1, 34.7912802696228
                    # rucklidge_attractor          m = 6, tau=1, 31.89549160003662
                    # seno                         m = 6, tau=1, 0.3272838592529297
                    # shawn_van_der_pol_oscillator m = 6, tau=1, 30.14395260810852
                    # simplest_cubic_flow          m = 6, tau=6, 32.612558364868164
                    # simplest_linear_flow         m = 6, tau=1, 29.60354256629944

m 6 tau 1 tiempo 32.949333906173706


In [21]:
def ob_SNR(xn,x=[],t=[],filename='SNR.csv',showPlot=0):
    """
    Parameters:
           xn.- Signal + noise
           x.- Original signal
           t.- time
           filename.- donde guardar los SNR obtenidos
           showPlot.- muestra las graficas con 1 y con 0 no
    Return:
           SNR.- Signal to noise ratio 
    """
    # Estimate noise using robust estimate
    beq = pyasl.BSEqSamp()
    # Define order of approximation (use larger values such as 2,3, or 4 for
    # faster varying or less well sampled data sets; also 0 is a valid order)
    N = 2
    # Define 'jump parameter' (use larger values such as 2,3, or 4 if correlation
    # between adjacent data point is suspected)
    j = 1
    nstd, nstdstd = beq.betaSigma(xn['col1'], N, j, returnMAD=True)
    # Obteniendo la varianza apartir de la desviacion estandar
    var_noise = np.power(nstd,2)
    # Ma va a contener la senal
    Ma = []
    # Res contiene los residuos los cuales se consideran como el ruido
    Res = []
    SNRcalculado = 0
    if len(x) == 0 and len(t) == 0:
        x = xn['col1']
        t = np.arange(len(xn))
    for window in range(2,20):
#         print("Ancho de ventana = ",window)
        # Moving Average Medias Moviles
        Ma = xn['col1'].rolling(window=window,center=True).mean()
        # Residuos de la Media movil
        Res = xn['col1'] - Ma
        # Quitando los 
        Res = Res.dropna()
        # Obteniendo la varianza del ruido
        varianza_Noise_sin = np.var(Res)      
#         restd = np.std(Ma)
        # Obteniendo las frecuencias y el espectro de potencia o densidad espectral de potencia
        fsenal, Pxx_den_senal = signal.periodogram(Ma[~np.isnan(Ma)])
        fnoise, Pxx_den_noise = signal.periodogram(Res[~np.isnan(Res)])
        # Quitando nans de la densidad espectral de potencia tanto para la senal como para el ruido
        Pxx_den_senal = Pxx_den_senal[~np.isnan(Pxx_den_senal)]
        Pxx_den_noise = Pxx_den_noise[~np.isnan(Pxx_den_noise)]

        #Si la diferencia entre la varianza de pyastronomy y la varianza del ruido extraido es menor a 1e-3
        #Podemos decir que se ha encontrado el SNR o que no contiene SNR
        if np.abs(var_noise - varianza_Noise_sin) < 1e-3:
            # Si la ventana es de 2 quiere decir que la serie no contiene ruido
            if window == 2:
                print("No contiene ruido")
                SNRcalculado = "No hay ruido"
                break
            # Esta x se utilizo para las pruebas
            if len(x) > 0:
                SNRcalculado = 10*np.log10(np.mean(Pxx_den_senal)/np.mean(Pxx_den_noise))
                print("\nSe redujo el ruido con una ventana de",window)
                print("SNR detectado",SNRcalculado)
#                 print("SNR mediaS/desvN",1/(np.mean(Ma)/np.std(Res)))
                if showPlot:
                    print("ventana =",window)
                    od.grafica2D(t,x,t,Ma,
                             title1='Original',title2='Ruido reducido')
                    od.grafica2D(t,xn['col1'],t,Ma,
                             title1='Ruido',title2='Ruido reducido')
            else:
                # Se calcula el SNR = Psignal/Pnoise donde P es la potencia media https://en.wikipedia.org/wiki/Signal-to-noise_ratio
                SNRcalculado = 10*np.log10(np.mean(Pxx_den_senal)/np.mean(Pxx_den_noise))
                print("\nSe redujo el ruido con una ventana de",window)
                print("SNR detectado",SNRcalculado)
#                 print("SNR mediaS/desvN",1/(np.mean(Ma)/np.std(Res)))
                if showPlot:
                    print("ventana =",window)
                    od.grafica2D(t,xn['col1'],t,Ma,
                             title1='St. con ruido',title2='St. con ruido reducido')
            break
        if window == 19:
            SNRcalculado = 10*np.log10(np.mean(Pxx_den_senal)/np.mean(Pxx_den_noise))
            if showPlot:
                print("ventana =",window)
                od.grafica2D(t,x,t,Ma,
                         title1='St. con ruido',title2='St. con ruido reducido')
    return SNRcalculado

In [22]:
ts = pd.read_csv('st perturbadas/rossler/rossler_0_0_25.csv')
#od.obtenerOutliers(windAristeoM10m,k=5,showPlot=1)
ob_SNR(ts,showPlot=1)

ventana = 19


Loading BokehJS ...

25.259034909697387